## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [58]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Iqaluit,CA,24.80,overcast clouds,63.7506,-68.5145,Frobisher Inn
1,5,Atuona,PF,80.11,overcast clouds,-9.8000,-139.0333,Villa Enata
2,6,New Norfolk,AU,57.99,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
3,7,Iracoubo,GF,78.82,overcast clouds,5.4802,-53.2011,Le Gite D Iracoubo
4,8,Port Alfred,ZA,73.00,broken clouds,-33.5906,26.8910,The Halyards Hotel


In [39]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 90


In [40]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.



preferred_cities_temp_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)] 
preferred_cities_temp_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iqaluit,CA,63.7506,-68.5145,24.80,86,90,17.27,overcast clouds
1,1,Pervomayskoye,RU,43.9190,46.7102,38.57,75,99,9.17,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,80.11,78,100,22.17,overcast clouds
6,6,New Norfolk,AU,-42.7826,147.0587,57.99,77,98,1.01,overcast clouds
7,7,Iracoubo,GF,5.4802,-53.2011,78.82,86,100,15.08,overcast clouds


In [59]:
# 4a. Determine if there are any empty rows.
preferred_cities_temp_df.count()

City_ID                539
City                   539
Country                538
Lat                    539
Lng                    539
Max Temp               539
Humidity               539
Cloudiness             539
Wind Speed             539
Current Description    539
dtype: int64

In [60]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_temp_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iqaluit,CA,63.7506,-68.5145,24.80,86,90,17.27,overcast clouds
1,1,Pervomayskoye,RU,43.9190,46.7102,38.57,75,99,9.17,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,80.11,78,100,22.17,overcast clouds
6,6,New Norfolk,AU,-42.7826,147.0587,57.99,77,98,1.01,overcast clouds
7,7,Iracoubo,GF,5.4802,-53.2011,78.82,86,100,15.08,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
675,675,Kota Kinabalu,MY,5.9749,116.0724,77.00,88,20,2.30,few clouds
677,677,Court-Saint-Etienne,BE,50.6167,4.5667,41.00,93,75,18.41,light rain
678,678,Jardim,BR,-21.4803,-56.1381,84.97,43,0,6.87,clear sky
679,679,Mekhe,SN,15.1097,-16.6218,77.00,41,0,6.91,clear sky


In [61]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Iqaluit,CA,24.80,overcast clouds,63.7506,-68.5145,
1,Pervomayskoye,RU,38.57,overcast clouds,43.9190,46.7102,
5,Atuona,PF,80.11,overcast clouds,-9.8000,-139.0333,
6,New Norfolk,AU,57.99,overcast clouds,-42.7826,147.0587,
7,Iracoubo,GF,78.82,overcast clouds,5.4802,-53.2011,
8,Port Alfred,ZA,73.00,broken clouds,-33.5906,26.8910,
9,Ushuaia,AR,53.60,few clouds,-54.8000,-68.3000,
10,Rikitea,PF,77.72,light rain,-23.1203,-134.9692,
11,Hobart,AU,57.99,overcast clouds,-42.8794,147.3294,
12,Hermanus,ZA,69.01,overcast clouds,-34.4187,19.2345,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
    
    hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [52]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df.count()

City                   490
Country                490
Max Temp               490
Current Description    490
Lat                    490
Lng                    490
Hotel Name             490
dtype: int64

In [68]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [69]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [70]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, -80), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))